In [16]:
import pandas as pd

from rolling_lookahead_dt_pulp import rollo_oct_pulp

from sklearn.model_selection import StratifiedKFold

import time
import os


In [17]:
# Load your training and test datasets
#train_data = pd.read_csv("datasets/example_datasets/train.csv")
#test_data = pd.read_csv("datasets/example_datasets/test.csv")

#stacked = pd.concat([train_data, test_data ], ignore_index=False)

#stacked.to_csv('datasets/example_datasets/stacked.csv', index=False)






In [18]:
target_label = "y"
depth_rolling_tree = 8
criterion_loss = "gini"
folds_cross_val = 10

In [19]:
# test, adult, car_evaluation, seismic
dataset_name = 'car_evaluation' # folder in 'results' will be named after it, results/dataset_name contains result .txt and .csv

#für andere Datensätze muss hier natürlich das ganze numerische, one hot encoding, binarize und, dass targets in erste col stehen davor

if dataset_name == 'test':
    data = pd.read_csv("datasets/example_datasets/stacked.csv")

if dataset_name == 'car_evaluation':
    data = pd.read_csv("datasets/car_evaluation/car_data_bin.csv")

if dataset_name == 'seismic':
    data = pd.read_csv("datasets/seismic_bumps/seismic_bin.csv")

dir_path = f'results/{dataset_name}'
#file_path = f'{dir_path}/output'

# Create the directory if it doesn't exist
#os.makedirs(dir_path, exist_ok=True)

In [20]:


features = data.drop(columns=['y'])
targets = data['y']

#print(features)
#print(targets)

skf = StratifiedKFold(n_splits=folds_cross_val, shuffle=True, random_state=42)

#scores = []

i=1 # index for fold number

for train_idx, test_idx in skf.split(features, targets): #gives row indices
    features_train = features.iloc[train_idx]
    features_test = features.iloc[test_idx]
    targets_train = targets.iloc[train_idx]
    targets_test = targets.iloc[test_idx]

    stacked_train = pd.concat([targets_train, features_train], axis=1, ignore_index=False)
    stacked_test = pd.concat([targets_test, features_test],axis=1, ignore_index=False)

    #stacked_train.to_csv(f'stacked_train_cv{i}.csv', index=False) # for getting csv files
    #stacked_test.to_csv(f'stacked_test_cv{i}.csv', index=False) # for getting csv files

    feature_columns = stacked_train.columns[1:] #assuming labels are in first column, ensured trough move_targets_to_front_and_rename()
    start_time_pulp = time.time()
    result_dict_pulp =rollo_oct_pulp.run(train=stacked_train, test=stacked_test, target_label="y", features=feature_columns, depth=depth_rolling_tree, criterion=criterion_loss)
    end_time_pulp = time.time()

    for depth in range(2,depth_rolling_tree+1):
        print(f'10 fold #{i}' + str(result_dict_pulp[depth]))

    print(f"Pulp execution time; 10 fold #{i}; for depth {depth_rolling_tree} : {end_time_pulp - start_time_pulp} seconds\n")

    # Create the directory if it doesn't exist
    os.makedirs(f'{dir_path}/pulp/fold{i}', exist_ok=True)

    for depth in range(2,depth_rolling_tree+1):
        with open(f'{dir_path}/pulp/fold{i}/fold{i}_acc_times_{dataset_name}.txt', 'a') as f:
            f.write(str(depth) + ': ' + str(result_dict_pulp[depth]) + "\n")

    with open(f'{dir_path}/pulp/fold{i}/fold{i}_acc_times_{dataset_name}.txt', 'a') as f:
            f.write(f"Pulp execution time for depth {depth_rolling_tree} : {end_time_pulp - start_time_pulp} seconds\n")

    for depth in range(2,depth_rolling_tree+1):
        with open(f'{dir_path}/pulp/fold{i}/depth{depth}_classification_{dataset_name}_test.csv', 'w') as f:
            f.write(str(result_dict_pulp['tree'][depth]['test'].to_csv()))
        with open(f'{dir_path}/pulp/fold{i}/depth{depth}_classification_{dataset_name}_train.csv', 'w') as f:
            f.write(str(result_dict_pulp['tree'][depth]['train'].to_csv()))
    
    i+=1











{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/20319a3a167f4474a9a1b484af49fe20-pulp.mps -sec 1800 -timeMode elapsed -branch -printingOptions all -solution /tmp/20319a3a167f4474a9a1b484af49fe20-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 28 COLUMNS
At line 4403 RHS
At line 4427 BOUNDS
At line 5310 ENDATA
Problem MODEL has 23 rows, 882 columns and 1764 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 1800
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.281509 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at r